# Tasks

* Implement vanilla autoencoder
* Train it on MNIST dataset MNIST
* Display digits recovered dy AE
* Display distribution of embeddings in latent space

#Dataset

http://yann.lecun.com/exdb/mnist/

The MNIST database of handwritten digits, has a training set of 60,000 examples, and a test set of 10,000 examples.

The images were centered in a 28x28 image by computing the center of mass of the pixels.



In [ ]:
import torch, torchvision
from torchvision.datasets import MNIST
from torchvision import transforms, utils


transf = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize( (0.1307) , (0.3081) )
                             ])

train_dataset = MNIST('mnist',
                      train = True,
                      download = True,
                      transform = transf
                      )

test_dataset = MNIST('mnist',
                     train = False,
                     download = True,
                     transform = transf
                     )

Display some samples along with corresponding labels

In [ ]:
import matplotlib.pyplot as plt
from torch.utils.data import  DataLoader

# Helper method
def show(grid):
  plt.axis("off")
  plt.imshow(grid.permute(1,2,0).numpy())
  plt.show()    

vis_dataloader = DataLoader(test_dataset, batch_size = 8, shuffle=True)
dataiter = iter(vis_dataloader)
example_batch = next(dataiter) # img1,  label

# display the data
grid = torchvision.utils.make_grid(example_batch[0])
show(grid)
print(example_batch[1].numpy())

# Model

Implement vanilla autoencoder model.

In [ ]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, latent_size):
        super().__init__()
        self.latent_size = latent_size
        self.encoder = nn.Sequential(nn.Linear(28 , 28), nn.Sigmoid())

    def forward(self,x):
        return self.encoder(x)  


class Decoder(nn.Module):
    def __init__(self, latent_size):
        super().__init__()
        self.latent_size = latent_size
        self.decoder = nn.Sequential(nn.Linear(28, 28))

    def forward(self,x):
        return self.decoder(x)   

##Smoke test 

In [ ]:
encoder = Encoder(2)
dummy = torch.randn((1,1,28,28))
print("Encoder In",dummy.shape)
embedding = encoder(dummy)
print("encoder Out",embedding.shape)


decoder = Decoder(2)
recovered = decoder(embedding)
print("Decoder out",recovered.shape)

assert( dummy.shape == recovered.shape) ,"Decoder out shape must be equal to input shape"

Encoder In torch.Size([1, 1, 28, 28])
encoder Out torch.Size([1, 1, 28, 28])
Decoder out torch.Size([1, 1, 28, 28])


###AE

Implement AutoEncoder class. 
Use Decoder and Encoder classes implemented early.

In [ ]:
class AutoEncoder(nn.Module):
  def __init__(self, latent_size):
    super().__init__()
    self.encoder = Encoder(latent_size)
    self.decoder = Decoder(latent_size)

  def forward(self, x):
    embedding = self.encoder(x)
    recovered = self.decoder(embedding)
    return recovered

# Train

Define dataloaders

In [ ]:
train_dataloader = DataLoader(train_dataset,shuffle=True,batch_size=128)
test_dataloader = DataLoader(test_dataset,shuffle=False,batch_size=256)

Define main training routine and train your model

In [ ]:
from torch import optim
from torch.utils.tensorboard import SummaryWriter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = AutoEncoder(2)
epochs = 20
outputs = []

for epoch in range(epochs):
    for (image, _) in train_dataloader:
        reconstructed = model.forward(image)
        outputs.append([image, reconstructed])
    print(f"Epoch = {epoch + 1}")

outputs = outputs[len(outputs) - 50:]

Epoch = 1
Epoch = 2
Epoch = 3
Epoch = 4
Epoch = 5
Epoch = 6
Epoch = 7
Epoch = 8
Epoch = 9
Epoch = 10
Epoch = 11
Epoch = 12
Epoch = 13
Epoch = 14
Epoch = 15
Epoch = 16
Epoch = 17
Epoch = 18
Epoch = 19
Epoch = 20


Tensorboard launching code



In [ ]:
import os
import shutil
import tensorflow as tf
import tensorboard as tb

#https://stackoverflow.com/questions/60730544/tensorboard-colab-tensorflow-api-v1-io-gfile-has-no-attribute-get-filesystem
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

# Helper method to run Tensorboard in Colab
def reinit_tensorboard(clear_log = True):
  # Directory for log files
  logs_base_dir = "runs"
  if clear_log:
    # Clear logs
    shutil.rmtree(logs_base_dir, ignore_errors = True)
    os.makedirs(logs_base_dir, exist_ok=True)
  # Colab magic
  %load_ext tensorboard
  %tensorboard --logdir {logs_base_dir}

reinit_tensorboard()

## Let's test the model

Compare original and recovered digits.

In [ ]:
length = len(outputs)

for image, reconstructed in outputs[:1]:
    image = image[:10]
    reconstructed = reconstructed[:10]

    for i, item in enumerate(image):
        item = item.reshape(-1, 28, 28)
        plt.imshow(item[0])
        plt.show()
    
    for i, item in enumerate(reconstructed):
        item = item.reshape(-1, 28, 28)
        plt.imshow(item[0].detach())
        plt.show()

# Conclusion

...

# Ideas for extra work

* Find the best latent space size
* Implement noise filtration with AE
* Test vector arithmetic in laent space
* Implemet VAE 
** Use Autoencoder class as base class
** Implement VAE Loss class
** Plot embeddings manifold in VAE latent space
** Compare decoding results VAE latent space with vanilla Autoencoder results
* Replace reconstruction loss from MSE to BCE
* Implement Conditional Autoencoder or CVAE